<a href="https://colab.research.google.com/github/lucas-bruzzone/Recipes-Dataset-64k-Dishes/blob/main/Dishes_Recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download the dataset


In [40]:
import os
import shutil

kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)

kaggle_json_src = '/content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/kaggle.json'
kaggle_json_dest = os.path.join(kaggle_dir, 'kaggle.json')
shutil.copy(kaggle_json_src, kaggle_json_dest)

os.chmod(kaggle_json_dest, 0o600)

In [41]:
dataset_name = "prashantsingh001/recipes-dataset-64k-dishes"
kagglehub.dataset_download(dataset_name)

Using Colab cache for faster access to the 'recipes-dataset-64k-dishes' dataset.


'/kaggle/input/recipes-dataset-64k-dishes'

In [42]:
import os
import shutil

source_dir = '/kaggle/input/recipes-dataset-64k-dishes'
destination_dir = '/content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/dataset'

os.makedirs(destination_dir, exist_ok=True)

# Copy the contents of the source directory to the destination directory
for item in os.listdir(source_dir):
    s = os.path.join(source_dir, item)
    d = os.path.join(destination_dir, item)
    # Use copy instead of move
    shutil.copy2(s, d)

print(f"Dataset copied from {source_dir} to {destination_dir}")

Dataset copied from /kaggle/input/recipes-dataset-64k-dishes to /content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/dataset


In [43]:
import os

dataset_dir = '/content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/dataset'
print(os.listdir(dataset_dir))

['1_Recipe_csv.csv', '2_Recipe_json.json']


In [44]:
import pandas as pd

dataset_path = '/content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/dataset/1_Recipe_csv.csv'
df = pd.read_csv(dataset_path)
display(df.head())

,recipe_title,category,subcategory,description,ingredients,directions,num_ingredients,num_steps
0,Air Fryer Potato Slices with Dipping Sauce,Air Fryer Recipes,Air Fryer Recipes,"These air fryer potato slices, served with a b...","[""3/4 cup ketchup"", ""1/2 cup beer"", ""1 tablesp...","[""Combine ketchup, beer, Worcestershire sauce,...",9,5
1,Gochujang Pork Belly Bites,Air Fryer Recipes,Air Fryer Recipes,These gochujang pork belly bites are sweet and...,"[""1 pound pork belly"", ""1/4 cup gochujang"", ""2...","[""Preheat an air fryer to 400 degrees F (200 d...",5,4
2,3-Ingredient Air Fryer Everything Bagel Chicke...,Air Fryer Recipes,Air Fryer Recipes,These 3-ingredient air fryer everything bagel ...,"[""1 \u00bc pounds chicken tenders"", ""1 tablesp...","[""Gather all ingredients. Preheat an air fryer...",3,4
3,Air Fryer Everything Bagel Chicken Cutlets,Air Fryer Recipes,Air Fryer Recipes,These air fryer everything bagel chicken cutle...,"[""4 chicken cutlets (about 1 pound total)"", ""s...","[""Preheat an air fryer to 400 degrees F (200 d...",9,9
4,Air Fryer Honey Sriracha Salmon Bites,Air Fryer Recipes,Air Fryer Recipes,These air fryer honey Sriracha salmon bites ar...,"[""1 tablespoon soy sauce"", ""1 tablespoon honey...","[""Preheat an air fryer to 400 degrees F (200 d...",5,5


# Import

In [45]:
import pandas as pd
import numpy as np
import json
import re
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# SETUP + EXPLORAÇÃO

In [46]:
print(f"\nTotal de receitas: {len(df)}")
print(f"Colunas: {df.columns.tolist()}")
print(f"\nCategorias únicas: {df['category'].nunique()}")
print(f"Subcategorias únicas: {df['subcategory'].nunique()}")


Total de receitas: 62126
Colunas: ['recipe_title', 'category', 'subcategory', 'description', 'ingredients', 'directions', 'num_ingredients', 'num_steps']

Categorias únicas: 267
Subcategorias únicas: 1032


In [47]:
# Estatísticas básicas
print("\n--- Estatísticas de Ingredientes e Passos ---")
print(df[['num_ingredients', 'num_steps']].describe())


--- Estatísticas de Ingredientes e Passos ---
       num_ingredients     num_steps
count     62126.000000  62126.000000
mean          9.017448      4.661865
std           3.830323      2.310253
min           1.000000      1.000000
25%           6.000000      3.000000
50%           9.000000      4.000000
75%          11.000000      6.000000
max          35.000000     25.000000


In [48]:
# Top 10 categorias
print("\n--- Top 10 Categorias ---")
print(df['category'].value_counts().head(10))



--- Top 10 Categorias ---
category
Main Dishes              3387
Healthy Recipes          2237
Appetizers And Snacks    2084
Cakes                    1954
Cookies                  1849
Beef Recipes             1400
Breads                   1352
Desserts                 1288
Breakfast And Brunch     1223
Pork                     1205
Name: count, dtype: int64


# PRÉ-PROCESSAMENTO

In [49]:
def parse_ingredients(ingredient_str):
    """Parse string JSON de ingredientes"""
    try:
        return json.loads(ingredient_str)
    except:
        return []

def normalize_ingredient(ingredient):
    """Normaliza um ingrediente removendo quantidades e formatando"""
    # Remove quantidades (números, frações, unidades)
    ingredient = re.sub(r'\d+[\s]*[/-]?[\s]*\d*', '', ingredient)

    # Remove unidades comuns
    units = ['cup', 'cups', 'tablespoon', 'tablespoons', 'teaspoon', 'teaspoons',
             'pound', 'pounds', 'ounce', 'ounces', 'gram', 'grams', 'kg', 'ml',
             'liter', 'liters', 'tbsp', 'tsp', 'oz', 'lb', 'g']

    for unit in units:
        ingredient = re.sub(rf'\b{unit}s?\b', '', ingredient, flags=re.IGNORECASE)

    # Remove parênteses e conteúdo
    ingredient = re.sub(r'\([^)]*\)', '', ingredient)

    # Limpa pontuação e espaços extras
    ingredient = re.sub(r'[,;]', '', ingredient)
    ingredient = ' '.join(ingredient.split())

    return ingredient.lower().strip()

In [50]:
# Parse e normaliza ingredientes
df['ingredients_list'] = df['ingredients'].apply(parse_ingredients)
df['ingredients_normalized'] = df['ingredients_list'].apply(
    lambda x: [normalize_ingredient(i) for i in x if normalize_ingredient(i)]
)

# Cria vocabulário de ingredientes
all_ingredients = []
for ingredients in df['ingredients_normalized']:
    all_ingredients.extend(ingredients)

ingredient_vocab = Counter(all_ingredients)

In [51]:
print(f"\nTotal de ingredientes únicos: {len(ingredient_vocab)}")
print(f"\n--- Top 20 Ingredientes Mais Comuns ---")
for ing, count in ingredient_vocab.most_common(20):
    print(f"{ing}: {count}")


Total de ingredientes únicos: 42697

--- Top 20 Ingredientes Mais Comuns ---
white sugar: 8651
salt: 8342
all-purpose flour: 7332
vanilla extract: 7027
½ salt: 6302
olive oil: 6180
water: 5229
large eggs: 4911
butter: 4666
baking powder: 4104
eggs: 3932
cloves garlic minced: 3904
½ white sugar: 3747
ground cinnamon: 3604
vegetable oil: 3585
baking soda: 3330
milk: 3190
¼ salt: 3152
salt and ground black pepper to taste: 3062
cooking spray: 2982


In [52]:
# Filtra ingredientes muito raros (aparecem menos de 5 vezes)
MIN_FREQUENCY = 5
filtered_vocab = {k: v for k, v in ingredient_vocab.items() if v >= MIN_FREQUENCY}
print(f"\nIngredientes após filtro (>={MIN_FREQUENCY}x): {len(filtered_vocab)}")


Ingredientes após filtro (>=5x): 13040


In [53]:
# Atualiza dataframe com ingredientes filtrados
df['ingredients_filtered'] = df['ingredients_normalized'].apply(
    lambda x: [i for i in x if i in filtered_vocab]
)

# CO-OCORRÊNCIAS

In [54]:
cooccurrence_matrix = defaultdict(lambda: defaultdict(int))

for ingredients in df['ingredients_filtered']:
    # Conta co-ocorrências de pares
    for ing1, ing2 in combinations(ingredients, 2):
        if ing1 != ing2:
            cooccurrence_matrix[ing1][ing2] += 1
            cooccurrence_matrix[ing2][ing1] += 1

print(f"\nMatriz de co-ocorrências construída")


Matriz de co-ocorrências construída


In [55]:
# Calcula métricas de associação
def calculate_association_metrics(ing1, ing2, cooccurrence_matrix, ingredient_vocab):
    """Calcula support, confidence e lift"""
    co_count = cooccurrence_matrix[ing1][ing2]
    ing1_count = ingredient_vocab[ing1]
    ing2_count = ingredient_vocab[ing2]
    total_recipes = len(df)

    support = co_count / total_recipes
    confidence = co_count / ing1_count if ing1_count > 0 else 0
    lift = (co_count * total_recipes) / (ing1_count * ing2_count) if ing1_count * ing2_count > 0 else 0

    return {
        'support': support,
        'confidence': confidence,
        'lift': lift,
        'count': co_count
    }

# Encontra combinações fortes
strong_associations = []

In [56]:
# Pega top ingredientes para análise (evita combinatória explosiva)
top_ingredients = [ing for ing, _ in ingredient_vocab.most_common(100)]

for ing1 in top_ingredients:
    for ing2, count in cooccurrence_matrix[ing1].items():
        if ing2 in top_ingredients and count >= 10:  # Threshold mínimo
            metrics = calculate_association_metrics(ing1, ing2, cooccurrence_matrix, ingredient_vocab)
            if metrics['lift'] > 1.2:  # Lift > 1 indica associação positiva
                strong_associations.append({
                    'ing1': ing1,
                    'ing2': ing2,
                    **metrics
                })

# Ordena por lift
strong_associations.sort(key=lambda x: x['lift'], reverse=True)

In [57]:
print(f"\nAssociações fortes encontradas: {len(strong_associations)}")
print("\n--- Top 15 Combinações com Maior Lift ---")
for assoc in strong_associations[:15]:
    print(f"{assoc['ing1']} + {assoc['ing2']}: lift={assoc['lift']:.2f}, count={assoc['count']}")


Associações fortes encontradas: 3680

--- Top 15 Combinações com Maior Lift ---
garlic powder + onion powder: lift=20.61, count=562
onion powder + garlic powder: lift=20.61, count=562
soy sauce + sesame oil: lift=19.43, count=395
sesame oil + soy sauce: lift=19.43, count=395
ground ginger + ground nutmeg: lift=19.17, count=156
ground nutmeg + ground ginger: lift=19.17, count=156
dried oregano + dried basil: lift=17.49, count=278
dried basil + dried oregano: lift=17.49, count=278
onion powder + cayenne pepper: lift=14.70, count=136
cayenne pepper + onion powder: lift=14.70, count=136
ground cumin + chili powder: lift=14.08, count=602
chili powder + ground cumin: lift=14.08, count=602
paprika + cayenne pepper: lift=14.01, count=157
cayenne pepper + paprika: lift=14.01, count=157
semisweet chocolate chips + chopped walnuts: lift=13.84, count=107


In [58]:
def generate_recipe_ingredients(base_ingredients, n_suggestions=5, category=None):
    """
    Gera sugestões de ingredientes adicionais baseado em co-ocorrências

    Args:
        base_ingredients: lista de ingredientes iniciais
        n_suggestions: número de sugestões
        category: categoria opcional para filtrar
    """
    suggestions = defaultdict(float)

    # Normaliza ingredientes de entrada
    base_normalized = [normalize_ingredient(ing) for ing in base_ingredients]

    # Filtra receitas por categoria se especificado
    recipes_subset = df if category is None else df[df['category'] == category]

    # Para cada ingrediente base, encontra co-ocorrências
    for base_ing in base_normalized:
        if base_ing in cooccurrence_matrix:
            for co_ing, count in cooccurrence_matrix[base_ing].items():
                if co_ing not in base_normalized:
                    # Score baseado em lift e frequência
                    metrics = calculate_association_metrics(base_ing, co_ing, cooccurrence_matrix, ingredient_vocab)
                    suggestions[co_ing] += metrics['lift'] * np.log(count + 1)

    # Ordena e retorna top N
    sorted_suggestions = sorted(suggestions.items(), key=lambda x: x[1], reverse=True)
    return [(ing, score) for ing, score in sorted_suggestions[:n_suggestions]]

In [59]:
test_cases = [
    (["chicken", "garlic"], None),
    (["chocolate", "butter"], "Desserts"),
    (["tomato", "basil", "mozzarella"], None)
]

In [60]:
for base, cat in test_cases:
    print(f"\n--- Ingredientes base: {base}" + (f" | Categoria: {cat}" if cat else "") + " ---")
    suggestions = generate_recipe_ingredients(base, n_suggestions=8, category=cat)
    for ing, score in suggestions:
        print(f"  • {ing} (score: {score:.2f})")


--- Ingredientes base: ['chicken', 'garlic'] ---
  • harissa paste (score: 1957.38)
  • alamata olives (score: 1468.04)
  • small red onion thinly sliced (score: 1304.92)
  • strip steak (score: 1113.15)
  • fettucine or other pasta (score: 1113.15)
  • packages cremini mushrooms (score: 1113.15)
  • container chive-and-onion cream cheese (score: 1113.15)
  • flat-leaf parsley (score: 1113.15)

--- Ingredientes base: ['chocolate', 'butter'] | Categoria: Desserts ---
  • pieces pre-cut refrigerated chocolate chip cookie dough (score: 11131.48)
  • scoop vanilla gelato (score: 11131.48)
  • candy-coated chocolate pieces (score: 5167.50)
  • ¼ superfine sugar (score: 1137.54)
  • whipped cream (score: 821.93)
  • ⅓ almond flour (score: 802.97)
  • ⅔ confectioners' sugar (score: 441.28)
  • sprinkles (score: 421.18)

--- Ingredientes base: ['tomato', 'basil', 'mozzarella'] ---
  • large basil leaves roughly torn (score: 9299.35)
  • can great northern beans (score: 7177.08)
  • oregano pr

# CORRELAÇÃO CATEGORIA

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Prepara documentos por categoria
category_documents = df.groupby('category')['ingredients_filtered'].apply(
    lambda x: ' '.join([' '.join(ingredients) for ingredients in x])
).to_dict()

print(f"\nCategorias analisadas: {len(category_documents)}")


Categorias analisadas: 267


In [62]:
# TF-IDF por categoria
def get_category_markers(category_documents, top_n=10):
    """Identifica ingredientes marcadores de cada categoria"""
    category_markers = {}

    for category, doc in category_documents.items():
        # TF-IDF simples: conta frequências
        words = doc.split()
        word_freq = Counter(words)
        total_words = len(words)

        # TF
        tf = {word: count / total_words for word, count in word_freq.items()}

        # IDF: quantas categorias têm esse ingrediente
        idf = {}
        for word in word_freq:
            categories_with_word = sum(1 for cat_doc in category_documents.values() if word in cat_doc)
            idf[word] = np.log(len(category_documents) / categories_with_word)

        # TF-IDF
        tfidf = {word: tf[word] * idf[word] for word in tf}

        # Top N marcadores
        top_markers = sorted(tfidf.items(), key=lambda x: x[1], reverse=True)[:top_n]
        category_markers[category] = top_markers

    return category_markers

category_markers = get_category_markers(category_documents, top_n=10)

In [63]:
# Mostra marcadores de categorias selecionadas
sample_categories = list(category_documents.keys())[:5]
print("\n--- Ingredientes Marcadores por Categoria (Top 5) ---")
for cat in sample_categories:
    print(f"\n{cat}:")
    for ing, score in category_markers[cat][:5]:
        print(f"  • {ing} (TF-IDF: {score:.4f})")


--- Ingredientes Marcadores por Categoria (Top 5) ---

Air Fryer Recipes:
  • garlic (TF-IDF: 0.0079)
  • panko (TF-IDF: 0.0061)
  • pepper (TF-IDF: 0.0060)
  • spray (TF-IDF: 0.0059)
  • paprika (TF-IDF: 0.0057)

Allrecipes Allstar Recipes:
  • / (TF-IDF: 0.0083)
  • fluid (TF-IDF: 0.0061)
  • extract (TF-IDF: 0.0054)
  • garlic (TF-IDF: 0.0051)
  • vanilla (TF-IDF: 0.0049)

Angel Food Cakes:
  • tartar (TF-IDF: 0.0612)
  • cake (TF-IDF: 0.0455)
  • whites (TF-IDF: 0.0417)
  • extract (TF-IDF: 0.0399)
  • vanilla (TF-IDF: 0.0276)

Antipasti:
  • toothpicks (TF-IDF: 0.0107)
  • cherry (TF-IDF: 0.0099)
  • tomatoes (TF-IDF: 0.0088)
  • prosciutto (TF-IDF: 0.0084)
  • capers (TF-IDF: 0.0078)

Appetizers And Snacks:
  • garlic (TF-IDF: 0.0054)
  • mayonnaise (TF-IDF: 0.0047)
  • pepper (TF-IDF: 0.0042)
  • cheese (TF-IDF: 0.0040)
  • popcorn (TF-IDF: 0.0039)


In [64]:
# FUNÇÃO GERADORA MELHORADA COM CATEGORIA
def generate_recipe_ingredients_v2(base_ingredients, n_suggestions=5, category=None):
    """
    Versão melhorada que considera marcadores de categoria
    """
    suggestions = defaultdict(float)
    base_normalized = [normalize_ingredient(ing) for ing in base_ingredients]

    # Score de co-ocorrências
    for base_ing in base_normalized:
        if base_ing in cooccurrence_matrix:
            for co_ing, count in cooccurrence_matrix[base_ing].items():
                if co_ing not in base_normalized:
                    metrics = calculate_association_metrics(base_ing, co_ing, cooccurrence_matrix, ingredient_vocab)
                    suggestions[co_ing] += metrics['lift'] * np.log(count + 1)

    # Boost se combina com categoria
    if category and category in category_markers:
        category_ings = {ing: score for ing, score in category_markers[category]}
        for ing in suggestions:
            if ing in category_ings:
                suggestions[ing] *= (1 + category_ings[ing])  # Boost proporcional ao TF-IDF

    sorted_suggestions = sorted(suggestions.items(), key=lambda x: x[1], reverse=True)
    return [(ing, score) for ing, score in sorted_suggestions[:n_suggestions]]

In [65]:
test_base = ["chicken", "soy sauce"]
test_category = "Asian Recipes"

print(f"\nIngredientes base: {test_base}")
print(f"Categoria: {test_category}\n")

print("V1 (sem boost de categoria):")
v1_suggestions = generate_recipe_ingredients(test_base, n_suggestions=8, category=test_category)
for ing, score in v1_suggestions:
    print(f"  • {ing} (score: {score:.2f})")

print("\nV2 (com boost de categoria):")
v2_suggestions = generate_recipe_ingredients_v2(test_base, n_suggestions=8, category=test_category)
for ing, score in v2_suggestions:
    print(f"  • {ing} (score: {score:.2f})")


Ingredientes base: ['chicken', 'soy sauce']
Categoria: Asian Recipes

V1 (sem boost de categoria):
  • harissa paste (score: 1957.38)
  • alamata olives (score: 1468.04)
  • small red onion thinly sliced (score: 1306.37)
  • / brown sugar (score: 978.69)
  • fresh oregano (score: 460.56)
  • carrots (score: 322.60)
  • large onion halved (score: 244.67)
  • very finely minced fresh ginger (score: 221.86)

V2 (com boost de categoria):
  • harissa paste (score: 1957.38)
  • alamata olives (score: 1468.04)
  • small red onion thinly sliced (score: 1306.37)
  • / brown sugar (score: 978.69)
  • fresh oregano (score: 460.56)
  • carrots (score: 322.60)
  • large onion halved (score: 244.67)
  • very finely minced fresh ginger (score: 221.86)


# EMBEDDINGS + SUBSTITUIÇÕES

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
# Cria contextos para cada ingrediente
def create_ingredient_contexts(df):
    """Cria contexto de cada ingrediente baseado em com quem aparece"""
    contexts = defaultdict(list)

    for ingredients in df['ingredients_filtered']:
        for ing in ingredients:
            # Contexto = outros ingredientes na mesma receita
            context = [i for i in ingredients if i != ing]
            contexts[ing].extend(context)

    return contexts

ingredient_contexts = create_ingredient_contexts(df)

In [68]:
# Cria embeddings simples baseados em co-ocorrência
def create_cooccurrence_embeddings(contexts, vocab, min_count=10):
    """Cria embeddings baseados em co-ocorrência"""
    # Filtra ingredientes com contexto suficiente
    valid_ingredients = [ing for ing in vocab if len(contexts[ing]) >= min_count]

    # Cria vetor de co-ocorrência para cada ingrediente
    embeddings = {}

    for ing in valid_ingredients:
        context_vector = Counter(contexts[ing])
        embeddings[ing] = context_vector

    return embeddings, valid_ingredients

embeddings, valid_ingredients = create_cooccurrence_embeddings(
    ingredient_contexts,
    filtered_vocab,
    min_count=10
)

print(f"\nEmbeddings criados para {len(embeddings)} ingredientes")



Embeddings criados para 13001 ingredientes


In [69]:
# Calcula similaridade entre ingredientes
def find_similar_ingredients(target_ingredient, embeddings, top_n=5):
    """Encontra ingredientes similares baseado em embeddings"""
    if target_ingredient not in embeddings:
        return []

    target_vector = embeddings[target_ingredient]
    similarities = []

    for ing, vector in embeddings.items():
        if ing != target_ingredient:
            # Similaridade de cosseno entre vetores de co-ocorrência
            common_contexts = set(target_vector.keys()) & set(vector.keys())
            if common_contexts:
                dot_product = sum(target_vector[ctx] * vector[ctx] for ctx in common_contexts)
                norm1 = np.sqrt(sum(v**2 for v in target_vector.values()))
                norm2 = np.sqrt(sum(v**2 for v in vector.values()))
                similarity = dot_product / (norm1 * norm2) if norm1 * norm2 > 0 else 0
                similarities.append((ing, similarity))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

In [70]:
# TESTE DE SUBSTITUIÇÕES
print("\n--- Teste de Substituições ---")
test_ingredients = ["chicken", "butter", "milk", "beef", "olive oil"]

for ing in test_ingredients:
    if ing in embeddings:
        print(f"\nSubstituições para '{ing}':")
        similar = find_similar_ingredients(ing, embeddings, top_n=5)
        for sim_ing, score in similar:
            print(f"  • {sim_ing} (similaridade: {score:.3f})")


--- Teste de Substituições ---

Substituições para 'chicken':
  • harissa paste (similaridade: 0.473)
  • / chicken broth (similaridade: 0.438)
  • cloves garlic (similaridade: 0.417)
  • dry bucatini pasta (similaridade: 0.416)
  • salt and freshly ground black pepper to taste (similaridade: 0.411)

Substituições para 'butter':
  • ¼ butter (similaridade: 0.877)
  • butter melted (similaridade: 0.859)
  • ½ butter (similaridade: 0.859)
  • milk (similaridade: 0.859)
  • vegetable oil (similaridade: 0.852)

Substituições para 'milk':
  • ½ milk (similaridade: 0.935)
  • ¾ milk (similaridade: 0.903)
  • baking soda (similaridade: 0.898)
  • butter melted (similaridade: 0.898)
  • ¼ milk (similaridade: 0.896)

Substituições para 'olive oil':
  • olive oil divided (similaridade: 0.817)
  • ground beef (similaridade: 0.803)
  • extra-virgin olive oil (similaridade: 0.802)
  • ½ olive oil (similaridade: 0.798)
  • ¼ olive oil (similaridade: 0.791)


In [71]:
# FUNÇÃO GERADORA FINAL COM SUBSTITUIÇÕES
def generate_recipe_final(base_ingredients, n_suggestions=5, category=None,
                         restrictions=None, allow_substitutions=True):
    """
    Versão final: co-ocorrências + categoria + substituições

    Args:
        base_ingredients: lista de ingredientes base
        n_suggestions: número de sugestões
        category: categoria opcional
        restrictions: lista de ingredientes a evitar
        allow_substitutions: se True, sugere substituições para restrições
    """
    suggestions = defaultdict(float)
    base_normalized = [normalize_ingredient(ing) for ing in base_ingredients]
    restrictions = [normalize_ingredient(r) for r in (restrictions or [])]

    # Verifica se algum ingrediente base está nas restrições
    substitutions_needed = {}
    if restrictions and allow_substitutions:
        for base_ing in base_normalized:
            if base_ing in restrictions:
                similar = find_similar_ingredients(base_ing, embeddings, top_n=3)
                if similar:
                    substitutions_needed[base_ing] = similar[0][0]  # Melhor substituto

    # Aplica substituições
    working_ingredients = base_normalized.copy()
    for original, substitute in substitutions_needed.items():
        working_ingredients.remove(original)
        working_ingredients.append(substitute)

    # Score de co-ocorrências
    for base_ing in working_ingredients:
        if base_ing in cooccurrence_matrix:
            for co_ing, count in cooccurrence_matrix[base_ing].items():
                if co_ing not in working_ingredients and co_ing not in restrictions:
                    metrics = calculate_association_metrics(base_ing, co_ing, cooccurrence_matrix, ingredient_vocab)
                    suggestions[co_ing] += metrics['lift'] * np.log(count + 1)

    # Boost de categoria
    if category and category in category_markers:
        category_ings = {ing: score for ing, score in category_markers[category]}
        for ing in suggestions:
            if ing in category_ings:
                suggestions[ing] *= (1 + category_ings[ing])

    sorted_suggestions = sorted(suggestions.items(), key=lambda x: x[1], reverse=True)

    return {
        'substitutions': substitutions_needed,
        'final_base': working_ingredients,
        'suggestions': [(ing, score) for ing, score in sorted_suggestions[:n_suggestions]]
    }


In [72]:
# TESTE FINAL
print("\n" + "=" * 60)
print("TESTE FINAL: GERAÇÃO COM SUBSTITUIÇÕES")
print("=" * 60)

result = generate_recipe_final(
    base_ingredients=["chicken", "butter", "garlic"],
    n_suggestions=8,
    category="Main Dishes",
    restrictions=["butter"],  # Sem manteiga
    allow_substitutions=True
)

print(f"\nIngredientes base: ['chicken', 'butter', 'garlic']")
print(f"Restrições: sem butter")
print(f"Categoria: Main Dishes\n")

if result['substitutions']:
    print("Substituições aplicadas:")
    for original, substitute in result['substitutions'].items():
        print(f"  • {original} → {substitute}")

print(f"\nIngredientes finais da base: {result['final_base']}")
print("\nSugestões de ingredientes adicionais:")
for ing, score in result['suggestions']:
    print(f"  • {ing} (score: {score:.2f})")


TESTE FINAL: GERAÇÃO COM SUBSTITUIÇÕES

Ingredientes base: ['chicken', 'butter', 'garlic']
Restrições: sem butter
Categoria: Main Dishes

Substituições aplicadas:
  • butter → ¼ butter

Ingredientes finais da base: ['chicken', 'garlic', '¼ butter']

Sugestões de ingredientes adicionais:
  • harissa paste (score: 1957.38)
  • alamata olives (score: 1468.04)
  • small red onion thinly sliced (score: 1307.76)
  • strip steak (score: 1113.15)
  • fettucine or other pasta (score: 1113.15)
  • packages cremini mushrooms (score: 1113.15)
  • container chive-and-onion cream cheese (score: 1113.15)
  • flat-leaf parsley (score: 1113.15)
